In [1]:
import pandas as pd
from scipy.stats import ttest_rel
import datetime

In [2]:
df = pd.read_csv('./Data/Police_Incidents_grid.csv')
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'])

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['Latitude_round'] = df['Latitude'].round(2)
df['Longitude_round'] = df['Longitude'].round(2)
df['grid'] = df['Latitude_round'].astype(str) + ',' + df['Longitude_round'].astype(str)

In [4]:
line_open = datetime.date(2016, 5, 20)
line_open_before = line_open - datetime.timedelta(days = 365)
line_open_after = line_open + datetime.timedelta(days = 365)

In [5]:
df_before = df[(df['Date Occurred'] >= line_open_before) & (df['Date Occurred'] < line_open)][['Date Occurred', 'grid']]
df_before = df_before.groupby(['grid', 'Date Occurred']).size().reset_index()
df_before.columns = ['Grid', 'Date', 'Count']
before_data = []
for grid in df['grid'].unique():
    for i in range(1, 366):
        date = line_open - datetime.timedelta(days = i)
        relevant_item = df_before[(df_before['Grid'] == grid) & (df_before['Date'] == date)]
        if relevant_item.shape[0] > 0:
            crimes = relevant_item.iloc[0]["Count"]
        else:
            crimes = 0
        before_data.append({"Grid":grid, "Date":date, "Count":crimes})
df_before = pd.DataFrame(before_data)

In [6]:
df_after = df[(df['Date Occurred'] <= line_open_after) & (df['Date Occurred'] > line_open)][['Date Occurred', 'grid']]
df_after = df_after.groupby(['grid', 'Date Occurred']).size().reset_index()
df_after.columns = ['Grid', 'Date', 'Count']
after_data = []
for grid in df['grid'].unique():
    for i in range(1, 366):
        date = line_open + datetime.timedelta(days = i)
        relevant_item = df_after[(df_after['Grid'] == grid) & (df_after['Date'] == date)]
        if relevant_item.shape[0] > 0:
            crimes = relevant_item.iloc[0]["Count"]
        else:
            crimes = 0
        after_data.append({"Grid":grid, "Date":date, "Count":crimes})
df_after = pd.DataFrame(after_data)

In [7]:
ttest_data = []
for grid in df['grid'].unique():
    before = df_before[df_before['Grid'] == grid]['Count'].values
    after = df_after[df_after['Grid'] == grid]['Count'].values
    ttest = ttest_rel(before, after)
    ttest_data.append({"Grid":grid, "Statistic":ttest.statistic, "P-Value":ttest.pvalue})
df_ttest = pd.DataFrame(ttest_data)

In [8]:
df_ttest.to_csv('./Data/grid-ttest.csv', index = False)